In [ ]:
# -----------------------------------------------
# タイタニックデータビューイング
# -----------------------------------------------

In [ ]:
# -------------------------------------------------------------------
# ライブラリの読込
# -------------------------------------------------------------------
import glob
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
import pandas as pd
from pandas import Series, DataFrame

import scipy as sp
import seaborn as sns
sns.set()

import sklearn
import unicodedata

%matplotlib inline
%precision 3

In [ ]:
plt.rcParams['font.family'] = 'IPAexGothic'

In [ ]:
print(mpl.get_configdir())
print(mpl.matplotlib_fname())

In [ ]:
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 300)

In [ ]:
pd.options.display.float_format = '{:.6f}'.format

In [ ]:
# -----------------------------------------------
# タイタニックデータ 読み込み
# -----------------------------------------------

In [ ]:
raw_path = '../data/raw/'

In [ ]:
# -------------------------------------------------------------------
# 学習データを読み込み
# -------------------------------------------------------------------
train = pd.read_csv(raw_path + 'train.csv')
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
train.head()

In [ ]:
# --------------------------------------------------------------------------------------
# 学習データ情報
# （欠損なし） PassengerId, Survived, Pclass, Name, Sex, SibSp, Parch, Ticket, Fare 
# （欠損あり） Age,Cabin,Embarked  
# --------------------------------------------------------------------------------------

In [ ]:
survived = train['Survived'].value_counts().rename({0:'死亡', 1:'生存'})
print(survived)

In [ ]:
plt.figure(figsize=(4, 2))
plt.title('生存者・死亡者数')
plt.bar(survived.index, survived)


In [ ]:
sex = train['Sex'].value_counts().rename({'male':'男性', 'female':'女性'})
print(sex)

In [ ]:
plt.figure(figsize=(4, 2))
plt.title('性別毎の乗客数')
plt.bar(sex.index, sex)


In [ ]:
pclass = train['Pclass'].value_counts().sort_index().rename({3:'三等', 2:'二等', 1:'一等'})
print(pclass)

In [ ]:
plt.figure(figsize=(4, 2))
plt.title('客室等級毎の乗客数')
plt.bar(pclass.index, pclass)

In [ ]:
age = train['Age'].value_counts().sort_index()
print(age)

In [ ]:
plt.figure(figsize=(4, 2))
plt.title('乗客の年齢分布')
plt.hist(train['Age'])

In [ ]:
# なぜか年齢が小数点の乗客が存在する

In [ ]:
plt.figure(figsize=(12, 3))
plt.title('年齢毎の乗客数')
plt.bar(age.index, age)

In [ ]:
plt.figure(figsize=(8, 3))
plt.title('乗客の運賃分布')
plt.hist(train['Fare'], bins=20)

In [ ]:
# 料金が0の謎の乗客が15名いる
train.loc[train['Fare']==0]

In [ ]:
train['Fare_log10'] = np.log10(train['Fare']+1)  # 0の客がいるので+1して対数を取る
plt.hist(train['Fare_log10'], bins=20)

In [ ]:
# -------------------------------------------------------------------
# 評価データを読み込み
# -------------------------------------------------------------------
test = pd.read_csv(raw_path + 'test.csv')
test.info()

In [ ]:
test.isnull().sum()

In [ ]:
# --------------------------------------------------------------------------------------
# 評価データ情報
# （欠損なし） PassengerId, Pclass, Name, Sex, SibSp, Parch, Ticket, Embarked
# （欠損あり） Age, Fare, Cabin
# --------------------------------------------------------------------------------------